In [4]:
import psycopg2 #importing required libraries

Inserting initial data into accounts DB.

In [5]:
def get_db_connection():
    connection = psycopg2.connect( database="postgres", 
                            user="postgres", 
                            password="admin", 
                            host="localhost", 
                            port="5432")
    return connection

In [6]:
con = get_db_connection()
cur = con.cursor()  

cur.execute('''
    INSERT INTO db3.account VALUES
    ('111', 'John Doe', 1000);
    INSERT INTO db3.account VALUES
    ('222', 'Merry Marry', 500);
    INSERT INTO db3.account VALUES
    ('333', 'Mr. X', 1500);
     ''')

con.commit()
con.close()

print('Initial data inserted!')

UniqueViolation: duplicate key value violates unique constraint "account_pkey"
DETAIL:  Key (account_id)=(111       ) already exists.


Happy case: transactions are executed for person which has enough money.

In [10]:
con = get_db_connection()
cur = con.cursor()  

try:
    cur.execute('''
        BEGIN;

        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f1', 'John Doe', 'LH1414', 'KBP', 'FRA', '03.01.2020');

        PREPARE TRANSACTION 'book_flight1';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h1', 'John Doe', 'Hotel Frankfurt', '03.01.2020', '03.03.2020');

        PREPARE TRANSACTION 'book_hotel1';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        UPDATE db3.account
        SET  ammount=ammount-600
        WHERE account_id='111';

        PREPARE TRANSACTION 'update_ammount1';
         ''')

    con.commit() 

except:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         ROLLBACK PREPARED 'book_flight1';
         ''')
    cur.execute('''
         ROLLBACK PREPARED 'book_hotel1';
         ''')
    print('Rollback executed!')

else:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         COMMIT PREPARED 'book_flight1';
         ''')
    cur.execute('''
         COMMIT PREPARED 'book_hotel1';
         ''')
    cur.execute('''
         COMMIT PREPARED 'update_ammount1';
         ''')
    print('Commit executed!')

con.close()

Bad case: Another transaction for same person end up in error after first one.

In [18]:
con = get_db_connection()
cur = con.cursor()  

try:
    cur.execute('''
        BEGIN;

        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f2', 'John Doe', 'LH1414', 'KBP', 'FRA', '04.01.2020');

        PREPARE TRANSACTION 'book_flight2';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h2', 'John Doe', 'Hotel Frankfurt', '04.01.2020', '04.01.2020');

        PREPARE TRANSACTION 'book_hotel2';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        UPDATE db3.account
        SET  ammount=ammount-600
        WHERE account_id='111';

        PREPARE TRANSACTION 'update_ammount2';
         ''')

    con.commit() 

except:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         ROLLBACK PREPARED 'book_flight2';
         ''')
    cur.execute('''
         ROLLBACK PREPARED 'book_hotel2';
         ''')
    print('Rollback executed!')

else:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         COMMIT PREPARED 'book_flight2';
         ''')
    cur.execute('''
         COMMIT PREPARED 'book_hotel2';
         ''')
    cur.execute('''
         COMMIT PREPARED 'update_ammount2';
         ''')
    print('Commit executed!')

con.close()

Rollback executed!


In such case no insert\updates happens since constraint is violated and insteed rollback is executed.

Now executing booking for person which initially has less than required on her account.

In [17]:
con = get_db_connection()
cur = con.cursor()  

try:
    cur.execute('''
        BEGIN;

        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f3', 'Merry Marry', 'LH1414', 'KBP', 'FRA', '03.11.2020');

        PREPARE TRANSACTION 'book_flight3';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h3', 'Merry Marry', 'Hotel Wiesbaden', '03.11.2020', '03.13.2020');

        PREPARE TRANSACTION 'book_hotel3';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        UPDATE db3.account
        SET  ammount=ammount-600
        WHERE account_id='222';

        PREPARE TRANSACTION 'update_ammount3';
         ''')

    con.commit() 

except:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         ROLLBACK PREPARED 'book_flight3';
         ''')
    cur.execute('''
         ROLLBACK PREPARED 'book_hotel3';
         ''')
    print('Rollback executed!')

else:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         COMMIT PREPARED 'book_flight3';
         ''')
    cur.execute('''
         COMMIT PREPARED 'book_hotel3';
         ''')
    cur.execute('''
         COMMIT PREPARED 'update_ammount3';
         ''')
    print('Commit executed!')

con.close()

Rollback executed!


As result we directly got rollback and no commit was executed.

And now executing two booking on person which have enough to cover both.

In [19]:
con = get_db_connection()
cur = con.cursor()  

try:
    cur.execute('''
        BEGIN;

        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f4', 'Mr. X', 'LH1414', 'KBP', 'FRA', '04.01.2020');

        PREPARE TRANSACTION 'book_flight4';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h4', 'Mr. X', 'Frankfurter Hof', '04.01.2020', '04.03.2020');

        PREPARE TRANSACTION 'book_hotel4';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        UPDATE db3.account
        SET  ammount=ammount-600
        WHERE account_id='333';

        PREPARE TRANSACTION 'update_ammount4';
         ''')

    con.commit() 

except:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         ROLLBACK PREPARED 'book_flight4';
         ''')
    cur.execute('''
         ROLLBACK PREPARED 'book_hotel4';
         ''')
    print('Rollback executed!')

else:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         COMMIT PREPARED 'book_flight4';
         ''')
    cur.execute('''
         COMMIT PREPARED 'book_hotel4';
         ''')
    cur.execute('''
         COMMIT PREPARED 'update_ammount4';
         ''')
    print('Commit executed!')

con.close()

Commit executed!


In [21]:
con = get_db_connection()
cur = con.cursor()  

try:
    cur.execute('''
        BEGIN;

        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f5', 'Mr. X', 'LH1414', 'KBP', 'FRA', '04.11.2020');

        PREPARE TRANSACTION 'book_flight5';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h5', 'Mr. X', 'Frankfurter Hof', '04.11.2020', '04.13.2020');

        PREPARE TRANSACTION 'book_hotel5';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        UPDATE db3.account
        SET  ammount=ammount-600
        WHERE account_id='333';

        PREPARE TRANSACTION 'update_ammount5';
         ''')

    con.commit() 

except:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         ROLLBACK PREPARED 'book_flight5';
         ''')
    cur.execute('''
         ROLLBACK PREPARED 'book_hotel5';
         ''')
    print('Rollback executed!')

else:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         COMMIT PREPARED 'book_flight5';
         ''')
    cur.execute('''
         COMMIT PREPARED 'book_hotel5';
         ''')
    cur.execute('''
         COMMIT PREPARED 'update_ammount5';
         ''')
    print('Commit executed!')

con.close()

UndefinedObject: prepared transaction with identifier "book_flight5" does not exist


Fail scenario: Executing for one of accounts same code but without commit prepared. Before executing it  - updating value not to violate constraint. 

In [22]:
con = get_db_connection()
cur = con.cursor()  

cur.execute('''
    UPDATE db3.account
    SET ammount=1000
    WHERE account_id='111';
     ''')

con.commit()
con.close()

print('Data updated!')

Data updated!


In [24]:
con = get_db_connection()
cur = con.cursor()  

cur.execute('''
    BEGIN;

    INSERT INTO db1.flight_booking(
    booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
    VALUES ('f6', 'John Doe', 'LH1414', 'KBP', 'FRA', '05.01.2020');

    PREPARE TRANSACTION 'book_flight6';
     ''')

con.commit() 

cur.execute('''
    BEGIN;

    INSERT INTO db2.hotel_booking(
    booking_id, client_name, hotel_name, arrival, departure)
    VALUES ('h6', 'John Doe', 'Hotel Frankfurt', '05.01.2020', '05.03.2020');

    PREPARE TRANSACTION 'book_hotel6';
     ''')

con.commit() 

cur.execute('''
    BEGIN;

    UPDATE db3.account
    SET  ammount=ammount-600
    WHERE account_id='111';

    PREPARE TRANSACTION 'update_ammount6';
     ''')

con.commit() 

con.close()

Implementing saga pattern.

In [ ]:
def book_flight(connection):
    
    cur = connection.cursor() 
    try:
        cur.execute('''
        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f7', 'John Doe', 'LH1414', 'KBP', 'FRA', '03.01.2020');
         ''')

        connection.commit()
        
        status = True
    
    except:
        status = False
    
    return status

def cancel_flight(connection):
    
    cur = connection.cursor()
    try:
        cur.execute('''
        DELETE FROM db1.flight_booking
        WHERE booking_id='f7';
         ''')
        
        connection.commit()
        
def book_hotel(connection):
    
    cur = connection.cursor() 
    try:
        cur.execute('''
        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h7', 'John Doe', 'Hotel Frankfurt', '03.01.2020', '03.03.2020');
         ''')

        connection.commit()
        
        status = True
    
    except:
        status = False
    
    return status


def cancel_hotel(connection):
    
    cur = connection.cursor()
    try:
        cur.execute('''
        DELETE FROM db1.flight_booking
        WHERE booking_id='f7';
         ''')
        
        connection.commit()

In [ ]:
con = get_db_connection()
cur = con.cursor()  

try:
    cur.execute('''
        BEGIN;

        INSERT INTO db1.flight_booking(
        booking_id, client_name, flight_number, from_airport, to_airport, flight_date)
        VALUES ('f1', 'John Doe', 'LH1414', 'KBP', 'FRA', '03.01.2020');

        PREPARE TRANSACTION 'book_flight1';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        INSERT INTO db2.hotel_booking(
        booking_id, client_name, hotel_name, arrival, departure)
        VALUES ('h1', 'John Doe', 'Hotel Frankfurt', '03.01.2020', '03.03.2020');

        PREPARE TRANSACTION 'book_hotel1';
         ''')

    con.commit() 

    cur.execute('''
        BEGIN;

        UPDATE db3.account
        SET  ammount=ammount-600
        WHERE account_id='111';

        PREPARE TRANSACTION 'update_ammount1';
         ''')

    con.commit() 

except:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         ROLLBACK PREPARED 'book_flight1';
         ''')
    cur.execute('''
         ROLLBACK PREPARED 'book_hotel1';
         ''')
    print('Rollback executed!')

else:
    con.set_isolation_level(0)

    cur = con.cursor()
    cur.execute('''
         COMMIT PREPARED 'book_flight1';
         ''')
    cur.execute('''
         COMMIT PREPARED 'book_hotel1';
         ''')
    cur.execute('''
         COMMIT PREPARED 'update_ammount1';
         ''')
    print('Commit executed!')

con.close()